<a href="https://colab.research.google.com/github/vinodkraman/RL4LLMs/blob/main/dpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install packages
!pip install numpy==1.26.3
!pip install torch==2.1.2
!pip install datasets==2.16.1
!pip install transformers==4.37.0
!pip install wandb==0.16.2
!pip install tqdm==4.66.1
!pip install trl==0.7.10

  Using cached torch-2.1.2-cp311-cp311-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl.m

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstalling tqdm-4.67.1:
      Successfully uninstalled tqdm-4.67.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.7.5 requires tqdm>=4.67, but you have tqdm 4.66.1 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 9.8 MB/s eta 0:00:00


In [9]:
#import packages
import argparse
import random
import numpy as np
from functools import partial

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW

from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

import wandb
from tqdm import tqdm

In [10]:
def seed_everything(seed=2003):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

In [11]:
def calculate_DPO_loss(model_preferred_logprob, model_dispreferred_logprob,
                       ref_preferred_logprob, ref_dispreferred_logprob,
                       beta=0.5):

    preferred_relative_logprob = model_preferred_logprob - ref_preferred_logprob
    dispreferred_relative_logprob = model_dispreferred_logprob - ref_dispreferred_logprob

    reward_accuracies = (preferred_relative_logprob > dispreferred_relative_logprob).float().mean()
    reward_margins = (preferred_relative_logprob - dispreferred_relative_logprob).mean()

    loss = -F.logsigmoid(beta * (preferred_relative_logprob - dispreferred_relative_logprob)).mean()

    return loss, preferred_relative_logprob.mean(), dispreferred_relative_logprob.mean(), reward_accuracies, reward_margins

In [12]:
def get_log_prob(logits, labels, prompt_lengths):
    log_probs = F.log_softmax(logits, dim=-1) #take softmax across each distribution over vocab space
    token_log_probs = torch.gather(log_probs, -1, labels.unsqueeze(-1)).squeeze(-1) #gets just the logits indexed at these tokens

    batch_size, seq_len = labels.shape
    response_mask = torch.arange(seq_len, device=labels.device).unsqueeze(0) >= prompt_lengths.unsqueeze(1)
    response_mask = response_mask.float()

    response_log_probs = (token_log_probs * response_mask).sum(dim=-1)
    response_lengths = response_mask.sum(dim=-1).clamp(min=1)
    return response_log_probs / response_lengths

In [13]:
def collate_fn(batch, tokenizer, max_length, device):
    prompt_encodings = tokenizer(
        ['Instruct: ' + item['prompt'] + '\n' for item in batch],
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

    chosen_encodings = tokenizer(
        ['Output: ' + item['chosen'] for item in batch],
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

    rejected_encodings = tokenizer(
        ['Output: ' + item['rejected'] for item in batch],
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

    prompt_preferred_ids = torch.cat([
        prompt_encodings.input_ids,
        chosen_encodings.input_ids
    ], dim=-1).to(device)

    prompt_dispreferred_ids = torch.cat([
        prompt_encodings.input_ids,
        rejected_encodings.input_ids
    ], dim=-1).to(device)

    prompt_preferred_mask = torch.cat([
        prompt_encodings.attention_mask,
        chosen_encodings.attention_mask
    ], dim=-1).to(device)

    prompt_dispreferred_mask = torch.cat([
        prompt_encodings.attention_mask,
        rejected_encodings.attention_mask
    ], dim=-1).to(device)

    prompt_lengths = prompt_encodings.attention_mask.sum(dim=-1) #length of each prompt (not including padding tokens)

    return {
        'prompt_preferred_ids': prompt_preferred_ids,
        'prompt_dispreferred_ids': prompt_dispreferred_ids,
        'prompt_preferred_mask': prompt_preferred_mask,
        'prompt_dispreferred_mask': prompt_dispreferred_mask,
        'prompt_lengths': prompt_lengths
    }

In [14]:
def train(model, ref_model, tokenizer, optimizer, train_dataloader, epochs=1, beta=0.1):
    model.train()
    ref_model.eval()

    for epoch in range(epochs):
        for batch in tqdm(train_dataloader):
            optimizer.zero_grad()

            model_preferred_logits = model(
                input_ids=batch['prompt_preferred_ids'],
                attention_mask=batch['prompt_preferred_mask']
            ).logits #shape (batch_size, sequence_length, vocab_size)

            model_preferred_logprob = get_log_prob(
                model_preferred_logits,
                batch['prompt_preferred_ids'],
                batch['prompt_lengths']
            )

            model_dispreferred_logits = model(
                input_ids=batch['prompt_dispreferred_ids'],
                attention_mask=batch['prompt_dispreferred_mask']
            ).logits

            model_dispreferred_logprob = get_log_prob(
                model_dispreferred_logits,
                batch['prompt_dispreferred_ids'],
                batch['prompt_lengths']
            )

            with torch.no_grad():
                ref_preferred_logits = ref_model(
                    input_ids=batch['prompt_preferred_ids'],
                    attention_mask=batch['prompt_preferred_mask']
                ).logits

                ref_preferred_logprob = get_log_prob(
                    ref_preferred_logits,
                    batch['prompt_preferred_ids'],
                    batch['prompt_lengths']
                )

                ref_dispreferred_logits = ref_model(
                    input_ids=batch['prompt_dispreferred_ids'],
                    attention_mask=batch['prompt_dispreferred_mask']
                ).logits

                ref_dispreferred_logprob = get_log_prob(
                    ref_dispreferred_logits,
                    batch['prompt_dispreferred_ids'],
                    batch['prompt_lengths']
                )

            loss, preferred_relative_logprob, dispreferred_relative_logprob, reward_accuracies, reward_margins = calculate_DPO_loss(
                model_preferred_logprob,
                model_dispreferred_logprob,
                ref_preferred_logprob,
                ref_dispreferred_logprob,
                beta=beta
            )

            loss.backward()
            optimizer.step()

            print(loss.item())

            wandb.log({
                'loss': loss.item(),
                'preferred_relative_logprob': preferred_relative_logprob.item(),
                'dispreferred_relative_logprob': dispreferred_relative_logprob.item(),
                'reward_accuracy': reward_accuracies.item(),
                'reward_margin': reward_margins.item()
            })

In [15]:
args = argparse.Namespace()
args.epochs = 10
args.beta = 0.10
args.batch_size = 8
args.max_length = 128
args.lr = 1e-6
args.seed = 2003
args.model_name = "EleutherAI/pythia-70m"
args.dataset_name = "jondurbin/truthy-dpo-v0.1"
wandb_project = "dpo"

In [17]:
seed_everything(args.seed)

wandb.login()
wandb.init(project=wandb_project, config=args)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(args.model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(args.model_name).to(device)
ref_model = AutoModelForCausalLM.from_pretrained(args.model_name).to(device)

ref_model.requires_grad_(False)

optimizer = AdamW(model.parameters(), lr=args.lr)

dataset = load_dataset(args.dataset_name, split="train")
collate = partial(collate_fn, tokenizer=tokenizer, max_length=args.max_length, device=device)
train_dataloader = DataLoader(dataset, batch_size=args.batch_size, shuffle=True, collate_fn=collate)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
train(model, ref_model, tokenizer, optimizer, train_dataloader, epochs=args.epochs, beta=args.beta)

model.save_pretrained("model-DPO")

  0%|          | 0/127 [00:00<?, ?it/s]